# convert from text file to excel file

In [8]:
import pandas as pd

# Read the comments from the text file
with open('reviewer2_comments.txt', 'r') as file:
    lines = file.readlines()

# Initialize lists to store parsed data
major_comments = []
minor_comments = []
current_section = None

# Define helper functions to identify specific and minor comments
def is_major_comment(line):
    line = line.strip()
    return line and line[0].isdigit() and line[1] == '.'

def is_minor_comment(line):
    return line.strip().startswith('- Line')

# Parse the comments
for line in lines:
    line = line.strip()
    if line.startswith("Specific comments:"):
        current_section = "Major Comments"
    elif line.startswith("Minor comments / technical suggestions:"):
        current_section = "Minor Comments"
    elif current_section == "Major Comments" and is_major_comment(line):
        major_comments.append(line)
    elif current_section == "Major Comments" and line:
        major_comments[-1] += " " + line
    elif current_section == "Minor Comments" and is_minor_comment(line):
        minor_comments.append(line)
    elif current_section == "Minor Comments" and line:
        minor_comments[-1] += " " + line

# Create DataFrames
df_major = pd.DataFrame({
    'Category': ['Major Comment'] * len(major_comments),
    'Comment': major_comments
})

df_minor = pd.DataFrame({
    'Category': ['Minor Comment'] * len(minor_comments),
    'Comment': minor_comments
})

# Concatenate the DataFrames
df = pd.concat([df_major, df_minor], ignore_index=True)

# Save the DataFrame to an Excel file
df.to_excel('reviewer2_comments.xlsx', index=False)

print("Comments have been successfully written to reviewer2_comments.xlsx")

Comments have been successfully written to reviewer2_comments.xlsx


# convert from excel file to word doc

In [2]:
%pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 26.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
Note: you may need to restart the kernel to use updated packages.


In [51]:
import openpyxl
from docx import Document
from docx.shared import RGBColor
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

xlsx_fname = 'point-to-point.xlsx'
docx_fname = 'point-to-point.docx'

# Load the workbook and select the active worksheet
wb = openpyxl.load_workbook(xlsx_fname)
ws = wb.active

# Define the color codes
seb_marcel_color = 'e06666'  # Pastel red
theo_color = 'f6b26b'  # Pastel orange

# Function to create a run with specified color
def create_colored_run(paragraph, text, color):
    run = paragraph.add_run(text)
    font = run.font
    font.color.rgb = RGBColor.from_string(color.upper())
    return run

# Function to apply blockquote style
def apply_blockquote(paragraph):
    p = paragraph._element
    pPr = p.get_or_add_pPr()
    pStyle = OxmlElement('w:pStyle')
    pStyle.set(qn('w:val'), 'Quote')
    pPr.insert(0, pStyle)

# Create a new Word document
doc = Document()

# Loop through the rows and process the comments and responses
current_reviewer = None
for row in ws.iter_rows(min_row=2):
    category = row[1].value
    comment = row[2].value
    response = row[3].value
    cell_content = row[0].value

    if category != current_reviewer:
        if category == 'Reviewer 1':
            doc.add_heading('Reviewer 1 Comments', level=1)
        elif category == 'Reviewer 2':
            doc.add_heading('Reviewer 2 Comments', level=1)
        current_reviewer = category

    # Create a paragraph for the comment and apply blockquote style
    comment_paragraph = doc.add_paragraph()
    apply_blockquote(comment_paragraph)
    comment_paragraph.add_run(comment)

    # Check the contents of the cell in column A and add the additional lines if needed
    if cell_content and ("sup" in cell_content.lower() or "supervisor" in cell_content.lower()):
        note_paragraph = doc.add_paragraph()
        create_colored_run(note_paragraph, "NB: Seb and Marcel to check.\n", seb_marcel_color)
        create_colored_run(note_paragraph, cell_content + "\n", seb_marcel_color)
    elif cell_content and cell_content.lower() not in ["yes", ""]:
        note_paragraph = doc.add_paragraph()
        create_colored_run(note_paragraph, "Theo to return to\n", theo_color)
        create_colored_run(note_paragraph, cell_content + "\n", theo_color)

    # Create a paragraph for the response
    response_paragraph = doc.add_paragraph()
    response_paragraph.add_run(response).font.color.rgb = RGBColor(0, 0, 255)

# Save the Word document
doc.save(docx_fname)